In [21]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch.cuda.amp import autocast

# ------------------- 参数配置 --------------------
esm_model_path="/public/home/kngll/llps/data/esm2_t36_3B_UR50D"
esm_weight_path="/public/home/kngll/Mambaphase/data/esm2_t36_3B_UR50D_mlm_finetuned.pth"
cls_model_path = "/public/home/kngll/Mambaphase/model/weights2/model_epoch_5.pth"
result_csv_path="/public/home/kngll/Mambaphase/results/predictions.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ESM_WEIGHT_PATH = "/public/home/kngll/Mambaphase/data/esm2_t36_3B_UR50D_mlm_finetuned.pth"
# ESM_MODEL_PATH = "/public/home/kngll/llps/data/esm2_t36_3B_UR50D"
# CLS_MODEL_PATH = "/public/home/kngll/Mambaphase/model/weights2/model_epoch_5.pth"
# RESULT_CSV_PATH = "/public/home/kngll/Mambaphase/results/predictions.csv"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
AA_LIST = "ACDEFGHIKLMNPQRSTVWYU"


amino_acid_to_index = {aa: idx for idx, aa in enumerate(AA_LIST)}

def infer_esm_rep(model, tokenizer, sequence, device):
    encoded_inputs = tokenizer(sequence, return_tensors='pt', padding=True, truncation=True)
    encoded_inputs = {k: v.to(device) for k, v in encoded_inputs.items()}
    with torch.no_grad():
        with autocast():
            outputs = model(**encoded_inputs, output_hidden_states=True)
    representations = outputs.hidden_states[-1]
    last_hidden_state = representations[:, 0, :]
    torch.cuda.empty_cache()
    return last_hidden_state.squeeze(0).cpu()

sequences = [
    " GHGVYGHGVYGHGPYGHGPYGHGLYW",
]



tokenizer = AutoTokenizer.from_pretrained(esm_model_path)
esm_model = AutoModelForMaskedLM.from_pretrained(esm_model_path)
esm_model.load_state_dict(torch.load(esm_weight_path), strict=False)
esm_model = esm_model.to(DEVICE)

print(f"共{len(sequences)}条序列，计算esm表示...")
esm_reps = []
for seq in sequences:
    if len(seq) > 4000:
        seq = seq[:4000]
    try:
        rep = infer_esm_rep(esm_model, tokenizer, seq, DEVICE)
        esm_reps.append(rep)
    except torch.cuda.OutOfMemoryError:
        print("OOM error! 忽略序列: ", seq[:10], "...")
        torch.cuda.empty_cache()
        esm_reps.append(torch.zeros(2560, dtype=torch.float))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_3506264/1621199238.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  esm_model.load_state_dict(torch.load(esm_weight_path), strict=False)
Asking to truncat

共1条序列，计算esm表示...


/tmp/ipykernel_3506264/1621199238.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


In [ ]:
# 已经有一个提取出来的特征列表“esm_reps ”，将/public/home/kngll/Mambaphase/model/phweight/best_model.pth模型导入。写出推断的代码，将结果保存到/public/home/kngll/Mambaphase/results/predictions.csv文件中。
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from torch import nn

# 定义模型架构（必须与训练时相同）
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=2560, num_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)

def predict_and_save(esm_reps, model_path, save_path):
    # 设置设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 确保保存目录存在
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    try:
        # 加载模型
        model = MLPClassifier().to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        
        # 转换输入数据
        if not isinstance(esm_reps, list):
            esm_reps = [esm_reps]
            
        # 预处理特征
        processed_features = []
        for feat in esm_reps:
            if isinstance(feat, np.ndarray):
                tensor_feat = torch.FloatTensor(feat)
            elif isinstance(feat, torch.Tensor):
                tensor_feat = feat.float()
            else:
                raise ValueError("Unsupported feature type")
                
            # 检查特征维度
            if tensor_feat.dim() == 1:
                tensor_feat = tensor_feat.unsqueeze(0)  # 添加batch维度
            elif tensor_feat.dim() != 2:
                raise ValueError(f"Invalid feature dimension: {tensor_feat.shape}")
                
            processed_features.append(tensor_feat)
            
        # 合并所有特征
        batch_data = torch.cat(processed_features).to(device)
        
        # 执行预测
        with torch.no_grad():
            outputs = model(batch_data)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
        
        # 转换为numpy
        preds_np = preds.cpu().numpy()
        probs_np = probs.cpu().numpy()
        
        # 创建结果DataFrame
        results = pd.DataFrame({
            "prediction": preds_np,
            "prob_low": probs_np[:, 0],
            "prob_mid": probs_np[:, 1],
            "prob_high": probs_np[:, 2]
        })
        
        # 保存结果
        results.to_csv(save_path, index=False)
        print(f"Successfully saved predictions to {save_path}")
        
        return results
    
    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        raise

# 使用示例
if __name__ == "__main__":
    # 假设 esm_reps 是预先加载的特征列表
    # 每个特征应为形状 (2560,) 的tensor或numpy数组
    
    # 模型路径
    MODEL_PATH = "/public/home/kngll/Mambaphase/model/phweight/best_model.pth"
    
    # 保存路径
    SAVE_PATH = "/public/home/kngll/Mambaphase/results/predictions.csv"
    
    # 执行预测
    predictions = predict_and_save(
        esm_reps=esm_reps,
        model_path=MODEL_PATH,
        save_path=SAVE_PATH
    )

Successfully saved predictions to /public/home/kngll/Mambaphase/results/predictions.csv


/tmp/ipykernel_3506264/1446016910.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [18]:
import os
import numpy as np
from Bio import SeqIO
from propy import PyPro
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 配置参数
DATA_DIR = "/public/home/kngll/Mambaphase/data/ph"  # 修改为实际路径
CLASS_MAP = {
    "high_ph": 0,
    "low_ph": 1,
    "neutral_ph": 2
}
VALID_AA = set("ACDEFGHIKLMNPQRSTVWY")  # 标准氨基酸字符

def validate_sequence(seq):
    """验证序列是否合法"""
    return all(aa in VALID_AA for aa in seq.upper()) and len(seq) >= 5

def extract_features(seq):
    """提取氨基酸组成（AAC）和二肽组成（DPC）特征"""
    try:
        descriptor = PyPro.GetProDes(seq)
        # 氨基酸组成（20维）
        aac = list(descriptor.GetAAComp().values())
        # 二肽组成（400维）
        dpc = list(descriptor.GetDPComp().values())
        return aac + dpc
    except Exception as e:
        print(f"特征提取失败: {str(e)}")
        return None

def load_dataset():
    """加载数据集并提取特征"""
    X, y = [], []
    
    for class_name, label in CLASS_MAP.items():
        fasta_path = os.path.join(DATA_DIR, f"{class_name}_sequences.fasta")
        if not os.path.exists(fasta_path):
            raise FileNotFoundError(f"文件不存在: {fasta_path}")
            
        print(f"正在处理: {class_name}")
        for record in SeqIO.parse(fasta_path, "fasta"):
            seq = str(record.seq).upper()
            if not validate_sequence(seq):
                continue
                
            features = extract_features(seq)
            if features is not None:
                X.append(features)
                y.append(label)
    
    return np.array(X), np.array(y)

def main():
    # 加载数据
    X, y = load_dataset()
    print(f"\n数据集信息:")
    print(f"- 总样本数: {X.shape[0]}")
    print(f"- 特征维度: {X.shape[1]}")
    print(f"- 类别分布: {np.bincount(y)}")

    # 划分训练测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # 训练模型
    clf = RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    )
    clf.fit(X_train, y_train)

    # 评估模型
    y_pred = clf.predict(X_test)
    print("\n分类报告:")
    print(classification_report(y_test, y_pred, target_names=CLASS_MAP.keys()))
    print("混淆矩阵:")
    print(confusion_matrix(y_test, y_pred))

if __name__ == "__main__":
    main()

正在处理: high_ph
正在处理: low_ph
正在处理: neutral_ph

数据集信息:
- 总样本数: 355
- 特征维度: 420
- 类别分布: [ 39  30 286]

分类报告:
              precision    recall  f1-score   support

     high_ph       0.25      0.12      0.17         8
      low_ph       0.00      0.00      0.00         6
  neutral_ph       0.82      0.96      0.89        57

    accuracy                           0.79        71
   macro avg       0.36      0.36      0.35        71
weighted avg       0.69      0.79      0.73        71

混淆矩阵:
[[ 1  0  7]
 [ 1  0  5]
 [ 2  0 55]]


/public/home/kngll/anaconda3/envs/deepstabp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/public/home/kngll/anaconda3/envs/deepstabp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/public/home/kngll/anaconda3/envs/deepstabp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [19]:
import os
import numpy as np
from Bio import SeqIO
from propy import PyPro
from sklearn.preprocessing import StandardScaler

def extract_features(seq):
    """提取氨基酸组成（AAC）和二肽组成（DPC）特征"""
    try:
        descriptor = PyPro.GetProDes(seq)
        aac = list(descriptor.GetAAComp().values())   # 20维
        dpc = list(descriptor.GetDPComp().values())    # 400维
        return aac + dpc
    except Exception as e:
        print(f"特征提取失败: {str(e)}")
        return None

def load_features(fasta_path):
    """加载FASTA文件并提取有效特征及对应序列"""
    features = []
    valid_records = []
    
    for record in SeqIO.parse(fasta_path, "fasta"):
        seq = str(record.seq).upper()
        # 过滤条件：长度≥10且只包含标准氨基酸
        if len(seq) < 10 or not set(seq).issubset("ACDEFGHIKLMNPQRSTVWY"):
            continue
            
        feat = extract_features(seq)
        if feat is not None:
            features.append(feat)
            valid_records.append(record)
    
    return np.array(features), valid_records

# 加载所有数据
high_path = "/public/home/kngll/Mambaphase/data/ph/high_ph_sequences.fasta"
low_path = "/public/home/kngll/Mambaphase/data/ph/low_ph_sequences.fasta"
neutral_path = "/public/home/kngll/Mambaphase/data/ph/neutral_ph_sequences.fasta"

high_features, high_records = load_features(high_path)
low_features, low_records = load_features(low_path)
neutral_features, neutral_records = load_features(neutral_path)

# 合并参考集（高+低pH）
reference_features = np.concatenate([high_features, low_features])

# 标准化特征
scaler = StandardScaler()
scaler.fit(reference_features)  # 仅用参考集计算标准化参数

neutral_scaled = scaler.transform(neutral_features)
reference_scaled = scaler.transform(reference_features)

# 计算马氏距离
def mahalanobis_distance(sample):
    ref_mean = np.mean(reference_scaled, axis=0)
    ref_cov = np.cov(reference_scaled, rowvar=False)
    delta = sample - ref_mean
    inv_cov = np.linalg.pinv(ref_cov)
    return np.sqrt(delta.T @ inv_cov @ delta)

distances = np.array([mahalanobis_distance(sample) for sample in neutral_scaled])

# 筛选距离最大的前40条
selected_indices = np.argsort(distances)[-40:][::-1]  # 降序排列取前40

# 生成新FASTA文件
output_path = "/public/home/kngll/Mambaphase/data/ph/selected_neutral.fasta"
selected_records = [neutral_records[i] for i in selected_indices]

with open(output_path, "w") as f:
    SeqIO.write(selected_records, f, "fasta")

# 打印验证信息
print(f"原始中性序列数: {len(neutral_records)} (已过滤无效序列)")
print(f"筛选后序列数: {len(selected_records)}")
print(f"最大马氏距离: {np.max(distances):.2f}")
print(f"最小选中距离: {np.min(distances[selected_indices]):.2f}")
print(f"结果已保存至: {output_path}")

原始中性序列数: 286 (已过滤无效序列)
筛选后序列数: 40
最大马氏距离: 1169.34
最小选中距离: 123.20
结果已保存至: /public/home/kngll/Mambaphase/data/ph/selected_neutral.fasta
